In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torchinfo

In [3]:
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import os
import csv
import time
import numpy as np

from tqdm import tqdm
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools import mask as coco_mask

import torchvision

from torchvision import datasets
from torchvision.transforms import functional as F
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader, RandomSampler, BatchSampler, SubsetRandomSampler

In [4]:
import torch
import torchvision
from torch import nn, Tensor

print(torch.__version__)
print(torchvision.__version__)

2.5.0+cu121
0.20.0+cu121


In [5]:
!git clone https://github.com/lkk688/DeepDataMiningLearning.git

Cloning into 'DeepDataMiningLearning'...
remote: Enumerating objects: 3542, done.
remote: Counting objects: 100% (806/806), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 3542 (delta 767), reused 702 (delta 702), pack-reused 2736 (from 1)
Receiving objects: 100% (3542/3542), 158.30 MiB | 20.83 MiB/s, done.
Resolving deltas: 100% (2509/2509), done.
Updating files: 100% (303/303), done.


In [6]:
%cd DeepDataMiningLearning

/content/DeepDataMiningLearning


In [7]:
!ls

dataapps		docker	images	notebooks	README.md	  sampledata  signalAI
DeepDataMiningLearning	docs	nlp	pyproject.toml	requirements.txt  scripts


In [8]:
!pip install flit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.3 MB/s eta 0:00:00


In [9]:
!flit install --symlink

>Fetching list of valid trove classifiers                                             I-flit.validate
Traceback (most recent call last):
  File "/usr/local/bin/flit", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/flit/__init__.py", line 216, in main
    installer = Installer.from_ini_path(
  File "/usr/local/lib/python3.10/dist-packages/flit/install.py", line 126, in from_ini_path
    return cls(ini_path.parent, ini_info, user=user, python=python,
  File "/usr/local/lib/python3.10/dist-packages/flit/install.py", line 112, in __init__
    raise RootInstallError
flit.install.RootInstallError: Installing packages as root is not recommended. To allow this, set FLIT_ROOT_INSTALL=1 and try again.


In [10]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

False
0


In [11]:
rootPath ='/content'
#Donload Kitti Dataset
mykitti = datasets.Kitti(root=rootPath, train= True, transform = None, target_transform = None, download = True)

100%|██████████| 12.6G/12.6G [07:02<00:00, 29.8MB/s]


Extracting /content/Kitti/raw/data_object_image_2.zip to /content/Kitti/raw


100%|██████████| 5.60M/5.60M [00:00<00:00, 6.39MB/s]


Extracting /content/Kitti/raw/data_object_label_2.zip to /content/Kitti/raw


In [12]:
from DeepDataMiningLearning.detection.dataset_kitti import KittiDataset, MyKittiDetection
import DeepDataMiningLearning.detection.transforms as T

In [13]:
def get_transformsimple(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ToDtype(torch.float, scale=True))
    # if train:
    #     transforms.append(RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [14]:
rootPath ='/content/Kitti'
is_train = True
kittidataset = MyKittiDetection(rootPath, train=True, transform=get_transformsimple(is_train))

In [15]:
print(kittidataset.INSTANCE_CATEGORY_NAMES)

['Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc', 'DontCare']


In [16]:
img, target = kittidataset[0]

In [17]:
print(img.shape) #CHW format

torch.Size([3, 375, 1242])


In [18]:
print(target.keys())

dict_keys(['boxes', 'labels', 'image_id', 'area', 'iscrowd'])


In [19]:
BATCH_SIZE=16
WORKERS = 2
SPLIT_RATE = 0.2  #20% for validation

In [20]:
dataset_size = len(kittidataset)
indices = list(range(dataset_size))
split = int(np.floor(SPLIT_RATE * dataset_size))

In [21]:
np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

In [22]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [23]:
train_sampler = torch.utils.data.RandomSampler(kittidataset)
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
train_batch_sampler = BatchSampler(train_sampler, BATCH_SIZE, drop_last=True)
val_batch_sampler = BatchSampler(val_sampler, 1, drop_last=True)

In [24]:
data_loader = torch.utils.data.DataLoader(
        kittidataset,
        batch_sampler=train_batch_sampler,
        num_workers=WORKERS,
        collate_fn=collate_fn
    )

val_loader = DataLoader(
    kittidataset,
    batch_sampler=val_batch_sampler,
    collate_fn=collate_fn,
    num_workers=WORKERS
)

In [25]:
# load a model pre-trained pre-trained
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 10  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# move model to the right device
model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 97.5MB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [26]:
model_path = '/content/final_model_3.pth'
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)

<ipython-input-26-63c831d504a4>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [27]:
def convert_to_coco_api(ds):#mykittidetectiondataset
    coco_ds = COCO()
    # annotation IDs need to start at 1, not 0, see torchvision issue #1530
    ann_id = 1
    dataset = {"images": [], "categories": [], "annotations": []}
    categories = set()
    ds_len=len(ds)
    print("convert to coco api:")
    progress_bar = tqdm(range(ds_len))
    for img_idx in range(ds_len):
        # find better way to get target
        # targets = ds.get_annotations(img_idx)
        img, targets = ds[img_idx] #img is [3, 1280, 1920],
        image_id = targets["image_id"].item() #need be int
        img_dict = {}
        img_dict["id"] = image_id
        img_dict["height"] = img.shape[-2] #img is CHW
        img_dict["width"] = img.shape[-1]
        dataset["images"].append(img_dict)
        bboxes = targets["boxes"].clone() #torch.Size([23, 4])
        bboxes[:, 2:] -= bboxes[:, :2] #[xmin, ymin, xmax, ymax] in torch to [xmin, ymin, width, height] in COCO
        bboxes = bboxes.tolist() #23 list of [536.0, 623.0, 51.0, 18.0]
        labels = targets["labels"].tolist() #torch.Size([23]) -> list 23 [1,1,1]
        areas = targets["area"].tolist() #torch.Size([23]) -> list 23 []
        iscrowd = targets["iscrowd"].tolist() #torch.Size([23]) -> list
        if "masks" in targets:
            masks = targets["masks"]
            # make masks Fortran contiguous for coco_mask
            masks = masks.permute(0, 2, 1).contiguous().permute(0, 2, 1)
        if "keypoints" in targets:
            keypoints = targets["keypoints"]
            keypoints = keypoints.reshape(keypoints.shape[0], -1).tolist()
        num_objs = len(bboxes)
        for i in range(num_objs):
            ann = {}
            ann["image_id"] = image_id
            ann["bbox"] = bboxes[i]
            ann["category_id"] = labels[i] #int
            categories.add(labels[i])
            ann["area"] = areas[i]
            ann["iscrowd"] = iscrowd[i]
            ann["id"] = ann_id
            if "masks" in targets:
                ann["segmentation"] = coco_mask.encode(masks[i].numpy())
            if "keypoints" in targets:
                ann["keypoints"] = keypoints[i]
                ann["num_keypoints"] = sum(k != 0 for k in keypoints[i][2::3])
            dataset["annotations"].append(ann)
            ann_id += 1
        progress_bar.update(1)
    dataset["categories"] = [{"id": i} for i in sorted(categories)]
    #print("convert_to_coco_api",dataset["categories"])
    coco_ds.dataset = dataset
    coco_ds.createIndex()
    return coco_ds

In [28]:
def convert_to_xywh(boxes):
    xmin, ymin, xmax, ymax = boxes.unbind(1)
    return torch.stack((xmin, ymin, xmax - xmin, ymax - ymin), dim=1)

In [29]:
def COCO_result(res):
      coco_results = []
      for original_id, prediction in res.items():
            if len(prediction) == 0:
                continue

            boxes = prediction["boxes"]
            boxes = convert_to_xywh(boxes).tolist()
            scores = prediction["scores"].tolist()
            labels = prediction["labels"].tolist()

            coco_results.extend(
                [
                    {
                        "image_id": original_id.item(),
                        "category_id": labels[k],
                        "bbox": box,
                        "score": scores[k],
                    }
                    for k, box in enumerate(boxes)
                ]
            )
      return coco_results

# 1 Prediction COCO Evaluation




In [32]:
model.eval()
with torch.no_grad():
    # Assuming test[0] is a tuple containing (images, targets)
    images, targets = next(iter(val_loader))
    images = list(image.to(device) for image in images)
    #targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    prediction = model(images) # Pass only the images to the model

prediction

[{'boxes': tensor([[602.1625, 166.9669, 632.6776, 238.6831],
          [205.2302, 152.0155, 242.9132, 252.1744],
          [182.2301, 151.4466, 215.2366, 245.9763],
          [809.9604, 148.3238, 862.7957, 261.3855],
          [483.7555, 162.2944, 510.4854, 214.7652],
          [566.4642, 160.6457, 593.3835, 177.6985],
          [782.6463, 155.6398, 800.2433, 207.6995],
          [688.5432, 158.2333, 699.7425, 193.8990],
          [ 21.6349, 154.2741, 115.1852, 337.5450],
          [655.1779, 163.1763, 665.8455, 197.7525],
          [529.9135, 166.7776, 549.8326, 209.1255],
          [730.1151, 155.4299, 743.1130, 192.7943],
          [741.8526, 155.3110, 753.6084, 190.3524],
          [666.4376, 160.5818, 675.6622, 196.3342],
          [737.3409, 154.8186, 747.6892, 191.9856],
          [  3.4735, 147.8808,  67.4592, 321.3450],
          [658.3141, 162.1800, 674.9164, 195.3184],
          [794.1062, 153.4664, 806.0146, 208.8812],
          [529.9193, 166.4106, 550.7820, 210.2693],
   

In [33]:
cocods = convert_to_coco_api(val_loader.dataset)

convert to coco api:


100%|██████████| 7481/7481 [02:57<00:00, 42.17it/s]

creating index...
index created!


In [34]:
outputs = [{k: v.to(device) for k, v in t.items()} for t in prediction]
res = {target["image_id"]: output for target, output in zip(targets, outputs)}

In [35]:
coco_res = COCO_result(res)

In [36]:
coco_dt = COCO.loadRes(cocods, coco_res)

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!


In [37]:
iou_type="bbox"
coco_eval = COCOeval(cocods, iouType=iou_type)

In [38]:
coco_eval.cocoDt = coco_dt

In [39]:
coco_eval.params.imgIds = list(np.unique(list(res.keys())))

In [40]:
print(cocods.getCatIds())
print([result["category_id"] for result in coco_res])

[1, 2, 3, 4, 5, 6, 7, 8, 9]
[4, 4, 4, 6, 6, 9, 4, 4, 4, 4, 8, 4, 4, 4, 4, 4, 4, 4, 6, 6, 4, 4, 4, 9, 4, 4, 4, 4, 8, 4, 4, 4, 4, 4, 4, 4, 4, 9, 4, 9, 9, 4]


In [41]:
list(np.unique(list(res.keys())))

[782]

In [42]:
# Check image ids in the ground truth (cocods)
image_ids_gt = cocods.getImgIds()
print("Image IDs in ground truth:", image_ids_gt)

Image IDs in ground truth: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 21

In [43]:
for prediction in coco_res:
    print(prediction)

{'image_id': 782, 'category_id': 4, 'bbox': [602.1625366210938, 166.96685791015625, 30.5150146484375, 71.71626281738281], 'score': 0.9978116154670715, 'segmentation': [[602.1625366210938, 166.96685791015625, 602.1625366210938, 238.68312072753906, 632.6775512695312, 238.68312072753906, 632.6775512695312, 166.96685791015625]], 'area': 2188.42281040363, 'id': 1, 'iscrowd': 0}
{'image_id': 782, 'category_id': 4, 'bbox': [205.2301788330078, 152.01553344726562, 37.68304443359375, 100.15890502929688], 'score': 0.9975816011428833, 'segmentation': [[205.2301788330078, 152.01553344726562, 205.2301788330078, 252.1744384765625, 242.91322326660156, 252.1744384765625, 242.91322326660156, 152.01553344726562]], 'area': 3774.2924686390907, 'id': 2, 'iscrowd': 0}
{'image_id': 782, 'category_id': 4, 'bbox': [182.23007202148438, 151.44662475585938, 33.00654602050781, 94.52970886230469], 'score': 0.9965957999229431, 'segmentation': [[182.23007202148438, 151.44662475585938, 182.23007202148438, 245.976333618

In [44]:
print(coco_dt.dataset["annotations"])

[{'image_id': 782, 'category_id': 4, 'bbox': [602.1625366210938, 166.96685791015625, 30.5150146484375, 71.71626281738281], 'score': 0.9978116154670715, 'segmentation': [[602.1625366210938, 166.96685791015625, 602.1625366210938, 238.68312072753906, 632.6775512695312, 238.68312072753906, 632.6775512695312, 166.96685791015625]], 'area': 2188.42281040363, 'id': 1, 'iscrowd': 0}, {'image_id': 782, 'category_id': 4, 'bbox': [205.2301788330078, 152.01553344726562, 37.68304443359375, 100.15890502929688], 'score': 0.9975816011428833, 'segmentation': [[205.2301788330078, 152.01553344726562, 205.2301788330078, 252.1744384765625, 242.91322326660156, 252.1744384765625, 242.91322326660156, 152.01553344726562]], 'area': 3774.2924686390907, 'id': 2, 'iscrowd': 0}, {'image_id': 782, 'category_id': 4, 'bbox': [182.23007202148438, 151.44662475585938, 33.00654602050781, 94.52970886230469], 'score': 0.9965957999229431, 'segmentation': [[182.23007202148438, 151.44662475585938, 182.23007202148438, 245.976333

In [45]:
coco_eval.evaluate()
print("Evaluation data:", coco_eval.evalImgs)

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.02s).
Evaluation data: [None, None, None, None, None, None, None, None, None, None, None, None, {'image_id': 782, 'category_id': 4, 'aRng': [0, 10000000000.0], 'maxDet': 100, 'dtIds': [1, 2, 3, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 39, 42], 'gtIds': [5299, 5300, 5301, 5303, 5306, 5307, 5308, 5309, 5310, 5311, 5312], 'dtMatches': array([[5303., 5300., 5301., 5307., 5308., 5299., 5309., 5311., 5312.,
           0.,    0.,    0.,    0.,    0., 5306.,    0.,    0.,    0.,
        5310.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.],
       [5303., 5300., 5301., 5307., 5308., 5299., 5309., 5311., 5312.,
           0.,    0.,    0.,    0.,    0., 5306.,    0.,    0.,    0.,
        5310.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
           0.,    0.,    0.,    0.],
       [5303., 5300., 5301., 530

In [46]:
coco_eval.accumulate()
coco_eval.summarize()

Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.700
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.982
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.909
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.622
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.821
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.518
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.470
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.697
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.722
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.644
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.842
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100

# 20 Prediction COCO Evaluation

In [45]:
cocods = convert_to_coco_api(val_loader.dataset)

convert to coco api:


100%|██████████| 7481/7481 [03:05<00:00, 40.31it/s]

creating index...
index created!


In [47]:
iou_type="bbox"
coco_eval = COCOeval(cocods, iouType=iou_type)

In [50]:
for i in tqdm(range(20)):
  model.eval()
  with torch.no_grad():
    images, targets = next(iter(val_loader))
    images = list(image.to(device) for image in images)
    prediction = model(images) # Pass only the images to the model

  outputs = [{k: v.to(device) for k, v in t.items()} for t in prediction]
  res = {target["image_id"]: output for target, output in zip(targets, outputs)}
  coco_result = COCO_result(res)


  coco_dt = COCO.loadRes(cocods, coco_result)
  coco_eval.cocoDt = coco_dt
  coco_eval.params.imgIds = list(np.unique(list(res.keys())))
  coco_eval.evaluate()
  coco_eval.accumulate()
#coco_eval.summarize()

  5%|▌         | 1/20 [00:06<02:03,  6.49s/it]

Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.04s).


 10%|█         | 2/20 [00:13<01:57,  6.54s/it]

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.02s).


 15%|█▌        | 3/20 [00:18<01:43,  6.07s/it]

...

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.00s).
Accumulating evaluation results...
DONE (t=0.02s).


 95%|█████████▌| 19/20 [01:56<00:06,  6.05s/it]

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.00s).
Accumulating evaluation results...
DONE (t=0.01s).


100%|██████████| 20/20 [02:03<00:00,  6.16s/it]

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.03s).


In [51]:
coco_eval.summarize()

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.860
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.981
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.889
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.502
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.921
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.775
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.675
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.879
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.879
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.500
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.925
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.775
